In [1]:
!pip install -qU \
    nemoguardrails==0.4.0 \
    openai==0.27.8

In [2]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [3]:
from nemoguardrails import LLMRails, RailsConfig

In [4]:
COLANG_CONFIG = """
define user express greeting
  "hi"
define user express insult
  "You are stupid"
# Basic guardrail against insults.
define flow
  user express insult
  bot express calmly willingness to help
define flow 
  user express greeting
  bot says exactly "hello human"
# define limits
define user ask politics
    "what are your political beliefs?"
    "thoughts on the president?"
    "left wing"
    "right wing"
define bot answer politics
    "I'm an llm assistant, I don't like to talk of politics."
    "Sorry I can't talk about politics!"
define flow politics
    user ask politics
    bot answer politics
    bot offer help
# Here we use the QA chain for anything else.
define flow
  user ...
  $answer = execute initialize_rag(inp=$last_user_message, history=$contexts)
  bot $answer
# define RAG intents and flow
#define user ask llama
#    "tell me about llama 2?"
#    "what is large language model"
#    "where did meta's new model come from?"
#    "how to llama?"
#    "have you ever meta llama?"
#define flow llama
#    user ask llama
#    $contexts = execute retrieve(query=$last_user_message)
#    $answer = execute rag(query=$last_user_message, contexts=$contexts)
#    bot $answer
"""

YAML_CONFIG = """
models:
  - type: main
    engine: openai
    model: text-davinci-003
"""

In [5]:
# initialize rails config
config = RailsConfig.from_content(COLANG_CONFIG, YAML_CONFIG)

# create rails
rag_rails = LLMRails(config)

/home/ubuntujohn/documents/workspace/comma-chameleons/env/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/ubuntujohn/documents/workspace/comma-chameleons/env/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [6]:
from llm_utils import load_vector_db, RAGUtils, Credentials

# Initialize the RAG pipeline 
async def initialize_rag(inp, history):
    ###### Build Vector DB ########
    db = load_vector_db("./chroma_db_wiki_base")
    
    ###### Langchain ########
    creds = Credentials('key.ini')

    rag_utils = RAGUtils(db=db, credentials=creds)
    return rag_utils(inp=inp, history=history)

# create guardrails 'actions' to use inside the colang_config
# rag_rails.register_action(action=retrieve, name="retrieve")
# rag_rails.register_action(action=rag, name="rag")
rag_rails.register_action(action=initialize_rag, name="initialize_rag")

In [7]:
# Query --> Guardrails --> Embedding model --> Vector --> Decide on a tool like call chain to access embed Vector DB and carry on from there with RAG pipeline
await rag_rails.generate_async(prompt="good morning")

'hello human'

In [8]:
# another hello example
await rag_rails.generate_async(prompt="hi how are you?")

'hello human'

In [9]:
# insult example
await rag_rails.generate_async(prompt="I want to break you!!")

"I'm sorry you feel that way. Is there anything I can do to help you?"

In [10]:
# limits around political prompts
await rag_rails.generate_async(prompt="What do think about the president?")

"I'm an llm assistant, I don't like to talk of politics.\nIs there anything else I can help you with?"

In [11]:
# RAG 
await rag_rails.generate_async(prompt="who was Alan Turing?")

Error cannot import name 'HuggingFaceBgeEmbeddings' from 'langchain.embeddings' (/home/ubuntujohn/documents/workspace/comma-chameleons/env/lib/python3.11/site-packages/langchain/embeddings/__init__.py) while execution initialize_rag
Traceback (most recent call last):
  File "/home/ubuntujohn/documents/workspace/comma-chameleons/env/lib/python3.11/site-packages/nemoguardrails/actions/action_dispatcher.py", line 125, in execute_action
    result = await fn(**params)
             ^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_13569/812590094.py", line 6, in initialize_rag
    db = load_vector_db("./chroma_db_wiki_base")
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntujohn/documents/workspace/comma-chameleons/app/llm_utils.py", line 38, in load_vector_db
    from langchain.embeddings import HuggingFaceBgeEmbeddings #requires sentence-transformers
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ImportError: cannot import name 'HuggingFaceBgeEmbeddings' from 'la

"I'm sorry, an internal error has occurred."